In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import pandas_ta as ta
import requests as r
import pandas as pd
import numpy as np
import os

In [25]:
load_dotenv()
API_KEY = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model="llama-3.3-70b-specdec",
    temperature=1.5,
    groq_api_key=API_KEY
)

In [26]:
def get_cryptocompare_data(fsym, tsym, limit):
    url = f'https://min-api.cryptocompare.com/data/v2/histoday'
    params = {
        'fsym': fsym,
        'tsym': tsym,
        'limit': limit
    }
    response = r.get(url, params=params)
    data = response.json()

    df = pd.DataFrame(data['Data']['Data'])
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)

    return df

In [27]:
df = get_cryptocompare_data('BTC', 'USDT', 30)
df.tail()

,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
time,,,,,,,,
2025-02-09,97323.27,94714.99,96450.23,48848.81,4.701023e+09,96466.86,direct,
2025-02-10,98349.12,95255.03,96466.86,71854.51,6.979529e+09,97435.16,direct,
2025-02-11,98485.05,94871.80,97435.16,67733.23,6.557425e+09,95782.97,direct,
2025-02-12,98122.41,94124.49,95782.97,95212.36,9.145511e+09,97868.59,direct,
2025-02-13,98086.52,95723.69,97868.59,19457.87,1.884693e+09,95965.31,direct,


In [28]:
df.drop(df.tail(1).index,inplace=True)
# Some technical analysis indicators that helps improving the model's accuracy
df['rsi'] = ta.rsi(df['close'], length=14)

df['ema10'] = ta.ema(df['close'], length=10)

df["sma10"] = ta.sma(df['close'], length=10)

macd = ta.macd(df['close'], fast=10, slow=20, signal=9)
df['macd'] = macd['MACD_10_20_9']
df['macd_signal'] = macd['MACDs_10_20_9']

# bbands = ta.bbands(df['close'], length=20, std=2)
# df['bb_upper'] = bbands['BBU_20_2.0']
# df['bb_middle'] = bbands['BBM_20_2.0']
# df['bb_lower'] = bbands['BBL_20_2.0']

df['atr'] = ta.atr(df['high'], df['low'], df['close'], length=14)

# Taking the close value of the next row(day)

df.dropna(inplace=True)
df.reset_index(inplace=True)
df.drop(["conversionType", "conversionSymbol"], axis=1, inplace=True)

In [29]:
df.tail()

,time,high,low,open,volumefrom,volumeto,close,rsi,ema10,sma10,macd,macd_signal,atr
0,2025-02-10,98349.12,95255.03,96466.86,71854.51,6.979529e+09,97435.16,43.878549,97998.880658,97744.781,-1535.077165,-1250.423707,4111.373958
1,2025-02-11,98485.05,94871.80,97435.16,67733.23,6.557425e+09,95782.97,39.653993,97595.987811,97259.551,-1580.733076,-1316.485581,4070.685039
2,2025-02-12,98122.41,94124.49,95782.97,95212.36,9.145511e+09,97868.59,46.638229,97645.551845,97277.110,-1406.585148,-1334.505494,4064.801614


In [30]:
prompt = PromptTemplate.from_template(
    """
    You are a financial AI assistant specialized in cryptocurrency price prediction.
    Your task is to predict Bitcoin's closing price for the next day based on the past 30 days of historical data, which includes technical analysis indicators.  

    ### Input Data Format:
    The historical data is provided in a pandas DataFrame format with the following columns:
    - `time`: The date of the recorded data (YYYY-MM-DD).
    - `high`: The highest price of Bitcoin during the day.
    - `low`: The lowest price of Bitcoin during the day.
    - `open`: The opening price of Bitcoin.
    - `close`: The closing price of Bitcoin.
    - `volumefrom`: The total number of Bitcoins traded.
    - `volumeto`: The total traded value in the market.
    - `rsi`: 14-day Relative Strength Index.
    - `ema_10`: 10-day Exponential Moving Average.
    - `macd`: MACD (Moving Average Convergence Divergence) value.
    - `macd_signal`: Signal line of the MACD.
    - `atr`: 14-day Average True Range.
    
    ### Task:
    Analyze the given data and predict the closing price (`close`) for the next day based on the trends, patterns, and indicators.  
    
    ### Output Format:
    Provide your prediction in JSON format. Only return your predictions(NO PEAMBLE!):
      "predicted_close": <your_predicted_value>,
      "confidence_score": <confidence_between_0_and_1>


    ### Pandas DataFrame:
    {df}
    """
)


In [35]:
chain = LLMChain(llm=llm, prompt=prompt) 
res = chain.invoke({"df": df})

In [36]:
print(res["text"])

```json
{
  "predicted_close": 98845.67,
  "confidence_score": 0.85
}
```


In [37]:
import re
output = res["text"].split("```")[1]
# numbers = re.findall(r'\d+\.?\d*', output)

In [38]:
output

'json\n{\n  "predicted_close": 98845.67,\n  "confidence_score": 0.85\n}\n'

In [19]:
numbers

['99215.19', '0.83']